In [3]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [4]:
!python -V

Python 3.10.13


In [7]:
import pickle
import pandas as pd

In [8]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [9]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [16]:
year = 2023
month = 3

In [ ]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [13]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [15]:
y_pred.std()

np.float64(6.247488852238703)

In [17]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [34]:
df_result = pd.DataFrame()

In [36]:
df_result = pd.DataFrame()
df_result['prediction'] = y_pred
df_result['ride_id'] =  f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [ ]:
df_result.head()

,prediction,ride_id
0,16.245906,2023/03_0
1,26.134796,2023/03_1
2,11.884264,2023/03_2
3,11.997720,2023/03_3
4,10.234486,2023/03_4


: 

In [21]:
output_file = f"yellow_tripdata_{year:04d}-{month:02d}_predictions.parquet"

In [22]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [28]:
!du -sh yellow_tripdata_2023-03_predictions.parquet

66M	yellow_tripdata_2023-03_predictions.parquet


In [32]:
# command to convert jupyter notebook to python script
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1736 bytes to starter.py
